In [164]:
import pandas as pd
from pandas import DataFrame
from pandas import Series

# Import dataset
services = pd.read_csv('Aurora_OfficeRonda_2_-_latest_version_-_False_-_2023-10-04-16-15-49.csv', sep=';', index_col = False)

# rename variables
newColumns = {'_coordenadas_latitude': 'lat', 
              '_coordenadas_longitude' : 'long',  
              'pais' : 'ubicacion' , 
              'punto_reporte' : 'lugar_enc', 
              'ubicacion' : 'punto_ayud', 
              'cualotropunto' : 'punto_ayud_otro', 
              'infraestructura' : 'cuenta_con' , 
              'sexo' : 'identifica', 
              'organizacionpertence' : 'org_pert', 
              'infraestructuradiscapacidad' : 'serv_disca' , 
              'servicios' : 'serv_tipo',
              'siserviciosnna' : 'serv_dif_n',
              'serviciosnna' : 'cual_serv1',
              'otroservicionna' : 'otro_cual_',
              'siserviciosmujeres' : 'serv_dif_m',
              'serviciosmujeres' : 'cual_ser_2',
              'otroserviciomujeres' : 'otro_dif_m',
              'retos' : 'princ_reto',
              'idiomas' : 'idioma_ent',
               'medios' :  'medios_bri',
               'diarioasesorialegal' : 'cap_serv_l', 
               'ayerasesorialegal' : 'cap_serv_1',
              'diarioalimentacionynutricion' : 'tipo_alime',
              'ayeralimentacionynutricion' : 'pers_alim',
              'diariosaludprimerosauxiliosyatencionmedica' : 'pers_atien',
              'ayersaludprimerosauxiliosyatencionmedica' : 'serv_sal_1',
              'diarioaguapotable' : 'prom_agua_',
              'ayeraguapotable' : 'serv_agua',
              'diariosaneamiento' : 'serv_san_d',
              'ayersaneamiento' : 'serv_san_a',
              'diariohigiene' : 'serv_hig_d',
              'ayerhigiene' : 'serv_hig_a',
              'diarioeducacion' : 'serv_edu_d',
              'ayereducacion' : 'serv_edu_a',
              'diarioalojamientotemporal' : 'serv_aloj',
              'ayeralojamientotemporal' : 'serv_aloj_',
              'diarioapoyooayudapsicosocial' : 'serv_psico',
              'ayerapoyooayudapsicosocial' : 'serv_psi_1',
              'diariotransportehumanitario' : 'serv_trans',
              'ayertransportehumanitario' : 'serv_tra_1',
              'diarioserviciosproteccion' : 'serv_prot_c', #new variable before was taken from "other" services
              'ayerserviciosproteccion' : 'serv_prot_a', #new variable before was taken from "other"  services
              'diariorestablecimientodecontactofamiliar' : 'serv_esta_c', #new variable before was taken from "other" services
              'ayerrestablecimientodecontactofamiliar' : 'serv_esta_a',  #new variable before was taken from "other" services
              'diariotransferenciasmonetarias' : 'serv_trans_c',
              'ayertransferenciasmonetarias' : 'serv_trans_a',
              'diariootro' : 'serv_tra_2',
              'ayerotro' : 'atend_serv',
              'organizacionpertence' : 'org_pert1',
              'organizacionprincipal' : 'org_pert2',
              'organizacionimplementadora' : 'org_pert_d',
              'capacidadsobrepasada' : 'sobrepasa_',
              'numeronna' : 'nna_atend',
              'cuantosnoacompanados' : 'cuan_nna_n',
              'cuantosseparados' : 'cuan_nna_s', 
              'nnaseparados' : 'nna_separ_', 
              'nnanoacompanados' : 'nna_no_aco', 
               }

services_carto = services.rename(columns=newColumns)

In [165]:
def codifyServices(value: str, values_dict: dict[str, int], otherValue:str):
    if(type(value) == float):
        return ""
    raw_values = value.split(" ")
    output = []
    for value in raw_values:
        try:      
            codedValue = values_dict[value]
            output.append(str(codedValue) )
        except Exception as e:
            output.append(otherValue)
            
    return "|".join(output)

def processColumn(dfColumn:Series, values_dict: dict[int, str], other_value: str):
    reversed_values_dict = dict( [( x[1],x[0])  for x in values_dict.items()])
    return dfColumn.apply(lambda x: codifyServices(x, reversed_values_dict, other_value))

def processMultValueColumns(df: DataFrame, columnObjectsList: list[dict]):
    """
    df: DataFrame object
    columnsObjectsList: list of column object
    columnObject: dictionary {"target_column": str, "output_column": str, values_dict: dict, other_value: str}

    return DataFrame Object
    """
    for columnObject in columnObjectsList:
        try:
            target_column = columnObject["target_column"]
            output_column = columnObject["output_column"]
            values_dict = columnObject["values_dict"]
            other_value = columnObject["other_value"]
            df[output_column] =  processColumn(df[target_column], values_dict, other_value)
        except Exception as e:
            print(e)
            continue
    return df

def exportToFile(df: DataFrame,fileType: str, exportName: str):
    """ 
    df -> Pandas DataFrame object
    fileType -> Either "csv" or "json"
    exportName -> File location
    """
    if(fileType == "csv"):
        name = f"{exportName}.csv"
        df.to_csv(name)
        print (f"data export to {name}")
    else:
        name = f"{exportName}.json"
        df.to_json(name, orient="records")
        print (f"data export to {name}")
    

In [166]:
# serv_tipo (separating by pipe symbol and categorized with number)

services_dict = {
  1 :"asesorialegal",
  2 :"alimentacionynutricion",
  3 :"saludprimerosauxiliosyatencionmedica",
  4 :"aguapotable",
  5 :"saneamiento",
  6 :"higiene",
  8 :"educacion",
  9 :"alojamientotemporal",
  11 :"acompañamientoayudapsicosocial",
  12 :"transportehumanitario",
  13 :"Otro",
  14 : "serviciosdeproteccion",
  16 : "restablecimientodecontactofamiliar",
  18 : "transferencias",  # new service: "Transferencias Monetarias"
      
}

# re structure variable cuenta_con
cuenta_con_dict = {
  1 :"accesoaserviciodeinternet",
  2 :"accesoaservicioelectrico",
  3 :"areadecomedorescocina",
  4 :"areaparadisposiciondebasuras",
  5 :"areascomunes",
  6 :"areasdelavanderia",
  7 :"areasderegistrodelapoblacion",
  8 :"ningunadelasanteriores",
}

# re structure variable children services (cual_serv1)
cual_serv1_dict = {
  1 :"nnaapoyooayudapsicosocial",
  2 :"nnagestiondecasos",
  3 :"nnaentregadekits",
  4 :"nnaalimentacionynutricion",
  5 :"nnabanosexclusivosnna", 
  6 :"nnacuidadosalternativos",
  7 :"nnaotro",
  8 :"espaciosamigables", # New option: Espacios Amigables
  9 : " ",
}

# re structure variable women services (cual_ser_2)
cual_ser_2_dict = {
  0 : " ",
  1 :"mujeresespaciosseguros",
  2 :"mujeresduchasobanos",
  3 :"mujeresgestiondecasosdeviolenciasbasadasengenero",
  4 :"mujeresproductosdehigienemenstrual",
  5 :"mujeresotro",
  6 :"mujerescontrolesprenatales", # New option: Controles prenatales para mujeres gestantes 
  7 :"mujeresapoyonutricionalgestantesylactantes", #New option: Apoyo nutricional para mujeres lactantes y/o gestantes

}

# re structure variable data storage (almacenamientoregistros)
registro_dict = {
  1 :"archivofisico",
  2 :"hojacalculo",
  3 :"software",
  4 :"otroalmacenarregistro",
  5 : " ",
}

# variable funding 
financ_dict = {
  1 :"organismosorganizacionesinternacionales",
  2 :"gobiernonacional",   
  3 :"empresasnacionales",
  4 :"empresasinternacionales",
  5 :"organismosnogubernamentaleslocalesonacionales",
  6 :"sociedadcivil",
  7 :"otrosfinanciamientos",

}

#  variable challenges
reto_dict = {
1 :"conseguirlafinanciacionnecesaria",
2 :"mejorarlacalidaddelosserviciosofrecidos",
3 :"ampliarymejorarlacolaboracionconotrasentidades",
4 :"mejorarlavisibilidaddeltrabajorealizado",
5 :"medirycomunicarelimpactosocialdelosserviciosprestados",
6 :"comunicacionconlosusuarios",
7 :"otroreto",
}

# variable lenguages
idio_dict = {
1 :"espanol",
2 :"ingles",
3 :"creole",
4 :"portugues",
5 :"otroidioma",
6 :"frances", 
}

# variable medios
medio_dict = {
1 :"caraacara",
2 :"folletos",
3 :"infografiaeninstalacionesdelpunto",
4 :"mediosaudiovisuales",
5 :"redessocialeseinternet",
6 :"otromedio"
}

values = [
    {
        "target_column": "serv_tipo",
        "output_column": "serv_tipo1",
        "values_dict": services_dict,
        "other_value": "0"
    },
    {
        "target_column": "cuenta_con",
        "output_column": "cuenta_c_1",
        "values_dict": cuenta_con_dict,
        "other_value": "0"
    },
    {
        "target_column": "cual_serv1",
        "output_column": "cual_ser_1",
        "values_dict": cual_serv1_dict,
        "other_value": "0"
    },
    {
        "target_column": "cual_ser_2",
        "output_column": "cual_ser_3",
        "values_dict": cual_ser_2_dict,
        "other_value": "0"
    },
    {
        "target_column": "almacenamientoregistros",
        "output_column": "almacenamientoregistros_",
        "values_dict": cual_ser_2_dict,
        "other_value": "0"
    },
     {
        "target_column": "financiamiento",
        "output_column": "financb",
        "values_dict": financ_dict,
        "other_value": "0"
    },
         {
        "target_column": "princ_reto",
        "output_column": "princ_re_1",
        "values_dict": reto_dict ,
        "other_value": "0"
    },
             {
        "target_column": "idioma_ent",
        "output_column": "idioma_e_1",
        "values_dict": idio_dict ,
        "other_value": "0"
    },

                 {
        "target_column": "medios_bri",
        "output_column": "medios_b_1",
        "values_dict": medio_dict ,
        "other_value": "0"
    },
]




In [128]:
output_df = processMultValueColumns(services_carto, values)
exportToFile(output_df, "csv", "output")

data export to output.csv


# daily report of surveys 

In [168]:
import datetime
 
services.fecha = pd.to_datetime(services.fecha)

In [169]:
# totals 
survey_counts = services.groupby('encuesta').organizacionprincipal.count().reset_index()
survey_counts = survey_counts.rename(columns={"organizacionprincipal": "total_encuestas"})
survey_counts

# totals by organization
unpivoted = services.groupby(['encuesta', 'fecha'])['pais'].count().reset_index()
pivoted = unpivoted.pivot(
    columns='fecha',
    index= 'encuesta',
    values='pais')
pivoted

fecha,2023-10-03,2023-10-04
encuesta,,
joseeduardoherrera,1.0,NaN
luisdavidcalderonpatino,1.0,NaN
luisricardosolermadrid,2.0,NaN
mariaalejandraolartedelgado,3.0,NaN
otro,1.0,1.0


In [170]:
#capacity vs attentions
services['servicios'] = services['servicios'].str.split(' ')
prueba = services.explode('servicios')

In [172]:
prueba= prueba[['_coordenadas_latitude', 
                '_coordenadas_longitude', 
                'servicios', 
                'organizacionprincipal', 
                'organizacionimplementadora',
                'pais',
                'punto_reporte', 
                '_uuid',
                'diarioalimentacionynutricion',
                'ayeralimentacionynutricion',
                'promalimentacionynutricion',
                'diarioalojamientotemporal',
                'ayeralojamientotemporal',
                'promalojamientotemporal',
                'diarioapoyooayudapsicosocial',
                'ayerapoyooayudapsicosocial',
                'promapoyooayudapsicosocial',
                'diarioasesorialegal',
                'ayerasesorialegal',
                'promasesorialegal',
                'diarioeducacion',
                'ayereducacion',
                'promeducacion',
                'diarioserviciosproteccion', 
                'ayerserviciosproteccion',
                'promserviciosproteccion',
                'diariorestablecimientodecontactofamiliar',
                'ayerrestablecimientodecontactofamiliar',
                'promrestablecimientodecontactofamiliar',
                'diariosaludprimerosauxiliosyatencionmedica',
                'ayersaludprimerosauxiliosyatencionmedica',
                'promsaludprimerosauxiliosyatencionmedica',
                'diarioaguapotable',
                'ayeraguapotable',
                'promaguapotable',
                'diariosaneamiento',
                'ayersaneamiento',
                'promsaneamiento',
                'diariohigiene',
                'ayerhigiene',
                'promhigiene',
                'diariotransportehumanitario',
                'ayertransportehumanitario',
                'promtransportehumanitario',
                'diariotransferenciasmonetarias',
                'ayertransferenciasmonetarias',
                'promtransferenciasmonetarias',
                'diariootro',
                'ayerotro',
                'promotro',
                'diariootro1',
                'ayerotro1',
                'promotro1', 
                ]].reset_index()
prueba

,index,_coordenadas_latitude,_coordenadas_longitude,servicios,organizacionprincipal,organizacionimplementadora,pais,punto_reporte,_uuid,diarioalimentacionynutricion,...,promtransportehumanitario,diariotransferenciasmonetarias,ayertransferenciasmonetarias,promtransferenciasmonetarias,diariootro,ayerotro,promotro,diariootro1,ayerotro1,promotro1
0,0,7.124690,-73.111897,acompañamientoayudapsicosocial,otra,otra,colombia,cucuta,af553f9e-b0a2-4d98-86ca-0ed2a7b4bb5e,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7.124690,-73.111897,alimentacionynutricion,otra,otra,colombia,cucuta,af553f9e-b0a2-4d98-86ca-0ed2a7b4bb5e,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,10.322561,-72.353604,acompañamientoayudapsicosocial,unicef,fundacionllovera,colombia,cucuta,77b990d1-8c10-44a1-b63c-ca5a69db3a37,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,10.322561,-72.353604,saludprimerosauxiliosyatencionmedica,unicef,fundacionllovera,colombia,cucuta,77b990d1-8c10-44a1-b63c-ca5a69db3a37,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.334659,-78.683263,alojamientotemporal,acnur,acnur,colombia,ipiales,1d9af2be-c301-42a0-857d-1061050c50ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1.334659,-78.683263,educacion,acnur,acnur,colombia,ipiales,1d9af2be-c301-42a0-857d-1061050c50ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,1.334659,-78.683263,serviciosdeproteccion,acnur,acnur,colombia,ipiales,1d9af2be-c301-42a0-857d-1061050c50ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,4.632828,-74.078760,acompañamientoayudapsicosocial,oim,oim,colombia,ipiales,9ff6f52c-7219-4ac1-80fe-bb81362431b8,250.0,...,300.0,400.0,400.0,400.0,NaN,NaN,NaN,NaN,NaN,NaN
8,8,4.632828,-74.078760,aguapotable,oim,oim,colombia,ipiales,9ff6f52c-7219-4ac1-80fe-bb81362431b8,250.0,...,300.0,400.0,400.0,400.0,NaN,NaN,NaN,NaN,NaN,NaN
9,9,4.632828,-74.078760,alimentacionynutricion,oim,oim,colombia,ipiales,9ff6f52c-7219-4ac1-80fe-bb81362431b8,250.0,...,300.0,400.0,400.0,400.0,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
prueba["capacidad"]=" "
prueba["ayer"]=" "
prueba["semana_pasada"]=" "


In [174]:
prueba.loc[(prueba['servicios']=='alimentacionynutricion'), 'capacidad'] = prueba['diarioalimentacionynutricion']
prueba.loc[(prueba['servicios']=='alojamientotemporal'), 'capacidad'] = prueba['diarioalojamientotemporal']
prueba.loc[(prueba['servicios']=='acompañamientoayudapsicosocial'), 'capacidad'] = prueba['diarioapoyooayudapsicosocial']
prueba.loc[(prueba['servicios']=='aguapotable'), 'capacidad'] = prueba['diarioaguapotable']
prueba.loc[(prueba['servicios']=='asesorialegal'), 'capacidad'] = prueba['diarioasesorialegal']
prueba.loc[(prueba['servicios']=='educacion'), 'capacidad'] = prueba['diarioeducacion']
prueba.loc[(prueba['servicios']=='higiene'), 'capacidad'] = prueba['diariohigiene']
prueba.loc[(prueba['servicios']=='restablecimientodecontactofamiliar'), 'capacidad'] = prueba[ 'diariorestablecimientodecontactofamiliar']
prueba.loc[(prueba['servicios']=='saludprimerosauxiliosyatencionmedica'), 'capacidad'] = prueba['diariosaludprimerosauxiliosyatencionmedica']
prueba.loc[(prueba['servicios']=='saneamiento'), 'capacidad'] = prueba['diariosaneamiento']
prueba.loc[(prueba['servicios']=='serviciosdeproteccion'), 'capacidad'] = prueba['diarioserviciosproteccion']
prueba.loc[(prueba['servicios']=='transferencias'), 'capacidad'] = prueba['diariotransferenciasmonetarias']
prueba.loc[(prueba['servicios']=='transportehumanitario'), 'capacidad'] = prueba['diariotransportehumanitario']
prueba.loc[(prueba['servicios']=='otro'), 'capacidad'] = prueba['diariootro']



In [175]:
prueba.loc[(prueba['servicios']=='alimentacionynutricion'), 'ayer'] = prueba['ayeralimentacionynutricion']
prueba.loc[(prueba['servicios']=='alojamientotemporal'), 'ayer'] = prueba['ayeralojamientotemporal']
prueba.loc[(prueba['servicios']=='acompañamientoayudapsicosocial'), 'ayer'] = prueba['ayerapoyooayudapsicosocial']
prueba.loc[(prueba['servicios']=='aguapotable'), 'ayer'] = prueba['ayeraguapotable']
prueba.loc[(prueba['servicios']=='asesorialegal'), 'ayer'] = prueba['ayerasesorialegal']
prueba.loc[(prueba['servicios']=='educacion'), 'ayer'] = prueba['ayereducacion']
prueba.loc[(prueba['servicios']=='higiene'), 'ayer'] = prueba['ayerhigiene']
prueba.loc[(prueba['servicios']=='restablecimientodecontactofamiliar'), 'ayer'] = prueba[ 'ayerrestablecimientodecontactofamiliar']
prueba.loc[(prueba['servicios']=='saludprimerosauxiliosyatencionmedica'), 'ayer'] = prueba['ayersaludprimerosauxiliosyatencionmedica']
prueba.loc[(prueba['servicios']=='saneamiento'), 'ayer'] = prueba['ayersaneamiento']
prueba.loc[(prueba['servicios']=='serviciosdeproteccion'), 'ayer'] = prueba['ayerserviciosproteccion']
prueba.loc[(prueba['servicios']=='transferencias'), 'ayer'] = prueba['ayertransferenciasmonetarias']
prueba.loc[(prueba['servicios']=='transportehumanitario'), 'ayer'] = prueba['ayertransportehumanitario']
prueba.loc[(prueba['servicios']=='otro'), 'ayer'] = prueba['ayerotro']

In [176]:
prueba.loc[(prueba['servicios']=='alimentacionynutricion'), 'semana_pasada'] = prueba['promalimentacionynutricion']
prueba.loc[(prueba['servicios']=='alojamientotemporal'), 'semana_pasada'] = prueba['promalojamientotemporal']
prueba.loc[(prueba['servicios']=='acompañamientoayudapsicosocial'), 'semana_pasada'] = prueba['promapoyooayudapsicosocial']
prueba.loc[(prueba['servicios']=='aguapotable'), 'semana_pasada'] = prueba['promaguapotable']
prueba.loc[(prueba['servicios']=='asesorialegal'), 'semana_pasada'] = prueba['promasesorialegal']
prueba.loc[(prueba['servicios']=='educacion'), 'semana_pasada'] = prueba['promeducacion']
prueba.loc[(prueba['servicios']=='higiene'), 'semana_pasada'] = prueba['promhigiene']
prueba.loc[(prueba['servicios']=='restablecimientodecontactofamiliar'), 'semana_pasada'] = prueba[ 'promrestablecimientodecontactofamiliar']
prueba.loc[(prueba['servicios']=='saludprimerosauxiliosyatencionmedica'), 'semana_pasada'] = prueba['promsaludprimerosauxiliosyatencionmedica']
prueba.loc[(prueba['servicios']=='saneamiento'), 'semana_pasada'] = prueba['promsaneamiento']
prueba.loc[(prueba['servicios']=='serviciosdeproteccion'), 'semana_pasada'] = prueba['promserviciosproteccion']
prueba.loc[(prueba['servicios']=='transferencias'), 'semana_pasada'] = prueba['promtransferenciasmonetarias']
prueba.loc[(prueba['servicios']=='transportehumanitario'), 'semana_pasada'] = prueba['promtransportehumanitario']
prueba.loc[(prueba['servicios']=='otro'), 'semana_pasada'] = prueba['promotro']

In [177]:
newColumns = { '_uuid' : 'submission_id',
               'servicios' : 'servicio',
               'semana_pasada' : 'semana+anterior',
               'ayer' : 'promedio+ayer',
               '_coordenadas_latitude' : 'observation_lat',
               '_coordenadas_longitude' : 'observation_lon',
               'punto_reporte' : 'Región'
               }

prueba = prueba.rename(columns=newColumns)

In [191]:
wide= prueba[[ 'submission_id',
                 'servicio',
                 'capacidad', 
                 'semana+anterior',
                 'promedio+ayer',
                 'observation_lat',
                 'observation_lon',
                 'Región',
                 'organizacionprincipal', 
                 'organizacionimplementadora',
                 
                ]].reset_index()

In [192]:
wide1= wide.dropna(subset=['servicio']).reset_index(drop=True)

wide1

,index,submission_id,servicio,capacidad,semana+anterior,promedio+ayer,observation_lat,observation_lon,Región,organizacionprincipal,organizacionimplementadora
0,0,af553f9e-b0a2-4d98-86ca-0ed2a7b4bb5e,acompañamientoayudapsicosocial,5.0,6.0,7.0,7.124690,-73.111897,cucuta,otra,otra
1,1,af553f9e-b0a2-4d98-86ca-0ed2a7b4bb5e,alimentacionynutricion,18.0,26.0,20.0,7.124690,-73.111897,cucuta,otra,otra
2,2,77b990d1-8c10-44a1-b63c-ca5a69db3a37,acompañamientoayudapsicosocial,10.0,12.0,11.0,10.322561,-72.353604,cucuta,unicef,fundacionllovera
3,3,77b990d1-8c10-44a1-b63c-ca5a69db3a37,saludprimerosauxiliosyatencionmedica,20.0,23.0,20.0,10.322561,-72.353604,cucuta,unicef,fundacionllovera
4,4,1d9af2be-c301-42a0-857d-1061050c50ff,alojamientotemporal,50.0,48.0,45.0,1.334659,-78.683263,ipiales,acnur,acnur
5,5,1d9af2be-c301-42a0-857d-1061050c50ff,educacion,70.0,70.0,70.0,1.334659,-78.683263,ipiales,acnur,acnur
6,6,1d9af2be-c301-42a0-857d-1061050c50ff,serviciosdeproteccion,30.0,35.0,25.0,1.334659,-78.683263,ipiales,acnur,acnur
7,7,9ff6f52c-7219-4ac1-80fe-bb81362431b8,acompañamientoayudapsicosocial,50.0,50.0,50.0,4.632828,-74.078760,ipiales,oim,oim
8,8,9ff6f52c-7219-4ac1-80fe-bb81362431b8,aguapotable,250.0,250.0,250.0,4.632828,-74.078760,ipiales,oim,oim
9,9,9ff6f52c-7219-4ac1-80fe-bb81362431b8,alimentacionynutricion,250.0,250.0,250.0,4.632828,-74.078760,ipiales,oim,oim


In [194]:
output_df = processMultValueColumns(wide1, values)
exportToFile(output_df, "csv", "bd_serv_prem_wide")

'serv_tipo'
'cuenta_con'
'cual_serv1'
'cual_ser_2'
'almacenamientoregistros'
'financiamiento'
'princ_reto'
'idioma_ent'
'medios_bri'
data export to bd_serv_prem_wide.csv
